# LMP Point-by-Point Optimization

The notebook optimizes the production or hydrogen and/or power for a given LMP for electricity point by point for a set of LMPs.  The results can be used to determine the optimal profit and operating schedule assuming there are no constraints linking one time point to another such as ramping limits or startup and shutdown considerations.

This notebook just does the optimization and stores the results. A companion notebook provides analysis. 

In [1]:
import os

from util.surrogate import SurrogateProcessModels
from util.data import get_model_data, DataObject
import pyomo.environ as pyo
import pandas as pd

In [2]:
data = DataObject()
surrogate = SurrogateProcessModels()
surrogate.generate_surrogate_models()

In [3]:
solver_obj = pyo.SolverFactory("ipopt")

In [4]:
def get_dataframe():
    return pd.DataFrame(
        columns=[
            "lmp ($/MWh)",
            "profit ($/hr)",
            "net_power (MW)",
            "h_prod (kg/s)",
            "ng_price ($/million BTU)",
            "h2_price ($/kg)",
            "el_revenue ($/hr)",
            "el_cost ($/hr)",
            "ng_cost ($/hr)",
            "h2_revenue ($/hr)",
            "other_cost ($/hr)",
            "fixed_cost ($/hr)",
            "mode_power_only",
            "mode_hydrogen_only",
            "mode_hydrogen",
            "mode_off",
        ]
    )
    
def add_data_row(*args, df, el_price, ng_price, h2_price, off):
    models = list(sorted(args, key=lambda x : pyo.value(x.profit), reverse=True))
    m = models[0]
    idx = len(df.index)    
    df.loc[idx, "lmp ($/MWh)"] = el_price
    df.loc[idx, "ng_price ($/million BTU)"] = ng_price
    df.loc[idx, "h2_price ($/kg)"] = h2_price
    df.loc[idx, "mode_power_only"] = 0
    df.loc[idx, "mode_hydrogen_only"] = 0 
    df.loc[idx, "mode_hydrogen"] = 0
    df.loc[idx, "mode_off"] = 0
    
    if off is not None and off >= pyo.value(m.profit):
        # the mode is off
        df.loc[idx, "mode_off"] = 1
        df.loc[idx, "h_prod (kg/s)"] = 0
        df.loc[idx, "net_power (MW)"] = 0
        df.loc[idx, "el_revenue ($/hr)"] = 0
        df.loc[idx, "el_cost ($/hr)"] = 0
        df.loc[idx, "ng_cost ($/hr)"] = 0
        df.loc[idx, "h2_revenue ($/hr)"] = 0
        df.loc[idx, "profit ($/hr)"] = off
        df.loc[idx, "other_cost ($/hr)"] = 0
        df.loc[idx, "fixed_cost ($/hr)"] = -off
    else:
        df.loc[idx, "profit ($/hr)"] = pyo.value(m.profit)
        df.loc[idx, "other_cost ($/hr)"] = pyo.value(m.other_cost)
        df.loc[idx, "fixed_cost ($/hr)"] = pyo.value(m.fixed_costs)
        # the mode is some model
        if hasattr(m, "net_power") and hasattr(m, "h_prod"):
            # hydrogen and power
            df.loc[idx, "mode_hydrogen"] = 1
            df.loc[idx, "net_power (MW)"] = pyo.value(m.net_power)
            df.loc[idx, "h_prod (kg/s)"] = pyo.value(m.h_prod)
            df.loc[idx, "el_revenue ($/hr)"] = pyo.value(m.net_power*m.el_price)
            df.loc[idx, "el_cost ($/hr)"] = pyo.value(-m.net_power*m.el_price)
            df.loc[idx, "h2_revenue ($/hr)"] = pyo.value(m.h_prod*m.h2_price)
            df.loc[idx, "ng_cost ($/hr)"] = pyo.value(m.fuel_cost)
        elif hasattr(m, "net_power"):
            # power only
            df.loc[idx, "mode_power_only"] = 1
            df.loc[idx, "net_power (MW)"] = pyo.value(m.net_power)
            df.loc[idx, "h_prod (kg/s)"] = 0
            df.loc[idx, "el_revenue ($/hr)"] = pyo.value(m.net_power*m.el_price)
            df.loc[idx, "el_cost ($/hr)"] = pyo.value(-m.net_power*m.el_price)
            df.loc[idx, "h2_revenue ($/hr)"] = 0
            df.loc[idx, "ng_cost ($/hr)"] = pyo.value(m.fuel_cost)
        elif hasattr(m, "h_prod"):
            # hydrogen only
            df.loc[idx, "fixed_cost ($/hr)"] = off
            df.loc[idx, "mode_hydrogen_only"] = 1
            df.loc[idx, "net_power (MW)"] = pyo.value(-m.electricity_cost/30.0)
            df.loc[idx, "h_prod (kg/s)"] = pyo.value(m.h_prod)
            df.loc[idx, "el_revenue ($/hr)"] = pyo.value(-m.electricity_cost/30.0*m.el_price)
            df.loc[idx, "el_cost ($/hr)"] = pyo.value(m.electricity_cost/30.0*m.el_price)
            df.loc[idx, "h2_revenue ($/hr)"] = pyo.value(m.h_prod*m.h2_price)
            df.loc[idx, "ng_cost ($/hr)"] = pyo.value(m.fuel_cost)

In [5]:
result_dir = "../lmp_point_by_point"
if not os.path.exists(result_dir):
    os.mkdir(result_dir)

In [6]:
h2_price = 2.0
groups = ["historical", "nrel", "o3"]

### Case 0 -- NGCC

In [7]:
model = "model0"

for data_set in data.metadata:
    if data.metadata[data_set]["group"] not in groups:
        continue
    lmps = data.data[data_set].dropna()
    ng_price = data.metadata[data_set]["ng_price"]
    result_file = os.path.join(result_dir, f"{data_set}_{model}.csv")
    m = surrogate.single_point_model(model, "power_only", ng_price=ng_price)
    m.ng_price.fix(ng_price)
    df = get_dataframe()
    for el_price in lmps:
        m.el_price.fix(el_price)
        res = solver_obj.solve(m)

        add_data_row(
            m, 
            df=df, 
            el_price=el_price, 
            ng_price=ng_price, 
            h2_price=h2_price,
            off=pyo.value(-m.fixed_costs)
        )
    df.to_csv(result_file)

### Case 1 -- SOFC

In [8]:
model = "model1"

for data_set in data.metadata:
    if data.metadata[data_set]["group"] not in groups:
        continue
    lmps = data.data[data_set].dropna()
    ng_price = data.metadata[data_set]["ng_price"]
    result_file = os.path.join(result_dir, f"{data_set}_{model}.csv")
    m = surrogate.single_point_model(model, "power_only", ng_price=ng_price)
    m.ng_price.fix(ng_price)
    df = get_dataframe()
    for el_price in lmps:
        m.el_price.fix(el_price)
        res = solver_obj.solve(m)

        add_data_row(
            m, 
            df=df, 
            el_price=el_price, 
            ng_price=ng_price, 
            h2_price=h2_price,
            off=pyo.value(-m.fixed_costs)
        )
    df.to_csv(result_file)

### Case 3 NGCC+SOEC

In [9]:
model = "model3"

for data_set in data.metadata:
    if data.metadata[data_set]["group"] not in groups:
        continue
    lmps = data.data[data_set].dropna()
    ng_price = data.metadata[data_set]["ng_price"]
    result_file = os.path.join(result_dir, f"{data_set}_{model}.csv")
    m = surrogate.single_point_model(model, "power_only", ng_price=ng_price)
    m2 = surrogate.single_point_model(model, "hydrogen", ng_price=ng_price)
    m.ng_price.fix(ng_price)
    m2.ng_price.fix(ng_price)
    m2.h2_price.fix(h2_price)
    df = get_dataframe()
    for el_price in lmps:
        m.el_price.fix(el_price)
        m2.el_price.fix(el_price)
        res = solver_obj.solve(m)
        res2 = solver_obj.solve(m2)
        add_data_row(
            m,
            m2,
            df=df, 
            el_price=el_price, 
            ng_price=ng_price, 
            h2_price=h2_price,
            off=pyo.value(-m.fixed_costs)
        )
    df.to_csv(result_file)

### Case 4 -- RSOC

In [10]:
model = "model4"

for data_set in data.metadata:
    if data.metadata[data_set]["group"] not in groups:
        continue
    lmps = data.data[data_set].dropna()
    ng_price = data.metadata[data_set]["ng_price"]
    result_file = os.path.join(result_dir, f"{data_set}_{model}.csv")
    m = surrogate.single_point_model(model, "power_only", ng_price=ng_price)
    m2 = surrogate.single_point_model(model, "hydrogen_only", ng_price=ng_price)
    m.ng_price.fix(ng_price)
    m2.ng_price.fix(ng_price)
    m2.h2_price.fix(h2_price)
    df = get_dataframe()
    for el_price in lmps:
        m.el_price.fix(el_price)
        m2.el_price.fix(el_price)
        res = solver_obj.solve(m)
        res2 = solver_obj.solve(m2)
        add_data_row(
            m,
            m2,
            df=df, 
            el_price=el_price, 
            ng_price=ng_price, 
            h2_price=h2_price,
            off=pyo.value(-m.fixed_costs)
        )
    df.to_csv(result_file)

### Case 5 SOFC+SOEC

In [11]:
model = "model5"

for data_set in data.metadata:
    if data.metadata[data_set]["group"] not in groups:
        continue
    lmps = data.data[data_set].dropna()
    ng_price = data.metadata[data_set]["ng_price"]
    result_file = os.path.join(result_dir, f"{data_set}_{model}.csv")
    m = surrogate.single_point_model(model, "power_only", ng_price=ng_price)
    m2 = surrogate.single_point_model(model, "hydrogen", ng_price=ng_price)
    m.ng_price.fix(ng_price)
    m2.ng_price.fix(ng_price)
    m2.h2_price.fix(h2_price)
    df = get_dataframe()
    for el_price in lmps:
        m.el_price.fix(el_price)
        m2.el_price.fix(el_price)
        res = solver_obj.solve(m)
        res2 = solver_obj.solve(m2)
        add_data_row(
            m,
            m2,
            df=df, 
            el_price=el_price, 
            ng_price=ng_price, 
            h2_price=h2_price,
            off=pyo.value(-m.fixed_costs)
        )
    df.to_csv(result_file)

KeyboardInterrupt: 

### Case 6 -- SOEC

In [ ]:
model = "model6"

for data_set in data.metadata:
    if data.metadata[data_set]["group"] not in groups:
        continue
    lmps = data.data[data_set].dropna()
    ng_price = data.metadata[data_set]["ng_price"]
    result_file = os.path.join(result_dir, f"{data_set}_{model}.csv")
    m = surrogate.single_point_model(model, "hydrogen_only", ng_price=ng_price)
    m.ng_price.fix(ng_price)
    m.h2_price.fix(h2_price)
    df = get_dataframe()
    for el_price in lmps:
        m.el_price.fix(el_price)
        res = solver_obj.solve(m)
        res2 = solver_obj.solve(m2)
        add_data_row(
            m,
            df=df, 
            el_price=el_price, 
            ng_price=ng_price, 
            h2_price=h2_price,
            off=pyo.value(-m.fixed_costs)
        )
    df.to_csv(result_file)